# Model Training & Evaluation

## Importing key functions

In [ ]:
from scripts import (
    load_and_split_data, 
    build_glm_pipeline,
    tune_glm_pipeline,
    build_lgbm_pipeline,
    tune_lgbm_pipeline,
    pipe_preprocessing,
    evaluate_model,
    plot_predicted_vs_actual_bar,
    feature_analysis)

## Initial stage (definitions and pipeline pre-processing)

In [ ]:
# Define features
numerical_features = ["Speed_limit", "Number_of_Casualties", "Number_of_Vehicles"]
categorical_features = [
        "Did_Police_Officer_Attend_Scene_of_Accident",
        "Junction_Control",
        "Junction_Detail",
        "Light_Conditions",
        "Pedestrian_Crossing-Human_Control",
        "Pedestrian_Crossing-Physical_Facilities",
        "Road_Type",
        "Urban_or_Rural_Area",
        "Weather_Conditions",
        "Road_Surface_Conditions",
        "Month",
        "Day_of_Week",
        "Hour_of_Day"
    ]

# Define target
target_column = "Accident_Severity"

In [ ]:
train_data, test_data = load_and_split_data()

In [ ]:
# Split train_data into features and target
X_train = train_data.drop(columns=[target_column,
                                   "Accident_Index"])
y_train = train_data[target_column]

# Split test_data into features and target
X_test = test_data.drop(columns=[target_column,
                                  "Accident_Index"])
y_test = test_data[target_column]

In [ ]:
preprocessor = pipe_preprocessing(numerical_features=numerical_features, categorical_features=categorical_features)

## GLM

In [ ]:
# Build and evaluate the pipeline
glm_pipeline, glm_accuracy = build_glm_pipeline(
        train_data=train_data,
        test_data=test_data,
        target_column=target_column,
        preprocessor=preprocessor
    )

In [ ]:
print(glm_pipeline)

In [ ]:
best_glm_pipeline, best_params, best_score = tune_glm_pipeline(glm_pipeline, train_data)

In [ ]:
glm_ap = evaluate_model(best_glm_pipeline, test_data=test_data)

In [ ]:
plot_predicted_vs_actual_bar(glm_ap)

Based on the confusion matrix and bar plot, our tuned GLM is overestimating the 'Slight' accident prevalence, and underestimating 'Serious' and 'Fatal' accident prevalence. 

In [ ]:
feature_analysis(best_glm_pipeline, X_train=X_train, y_train=y_train, top_n=5)

## LGBM

In [ ]:
lgbm_pipeline, lgbm_accuracy = build_lgbm_pipeline(
        train_data=train_data,
        test_data=test_data,
        target_column=target_column,
        preprocessor=preprocessor
    )

In [ ]:
best_lgbm_pipeline, best_params = tune_lgbm_pipeline(lgbm_pipeline, train_data)

In [ ]:
lgbm_ap = evaluate_model(best_lgbm_pipeline, test_data=test_data)

In [ ]:
plot_predicted_vs_actual_bar(lgbm_ap)

In [ ]:
top_feature_names = feature_analysis(best_lgbm_pipeline, X_train=X_train, y_train=y_train, top_n=5)

## PDP

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline

lgbm_classifier = LGBMClassifier(
    objective="multiclass",
    random_state=42,
    num_leaves=31,
    learning_rate=0.01,
    min_child_weight=10,
    n_estimators=500
)

lgbm_pdp = Pipeline(
        steps=[("preprocessor", preprocessor), ("classifier", lgbm_classifier)]
    )

lgbm_pdp.fit(X_train, y_train)

In [ ]:
import dalex as dx
lgbm_pdp_exp = dx.Explainer(lgbm_pdp, X_train, y_train, label="Tuned LGBM Pipeline")

pdp_lgbm = lgbm_pdp_exp.model_profile(variables= top_feature_names)
pdp_lgbm.plot()
